In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import keras
from keras.optimizers import Adam
from keras.layers import Input, GRU, Flatten, MaxPool2D, MaxPool1D
from keras.layers import PReLU, Dropout, Lambda, Dense
from keras.models import Model
import tensorflow as tf

from music_generator.basic.random import generate_dataset
from music_generator.basic.signalproc import SamplingInfo
from music_generator.musical.timing import Tempo
from music_generator.musical.scales import GenericScale
from music_generator.analysis.play import play_mono_as_stereo, play_array
from music_generator.basic.signalproc import mix_at
from music_generator.analysis import preprocessing

from music_generator.musical import scales
import numpy as np
from multiprocessing import Pool
from functools import partial

import matplotlib.pyplot as plt
from IPython.display import Audio
%matplotlib inline
import matplotlib

from scipy.io.wavfile import read
import pandas as pd

In [ ]:
matplotlib.rcParams['figure.figsize'] = (16.0, 12.0)
matplotlib.rcParams['lines.linewidth'] = 2
matplotlib.rcParams['axes.linewidth'] = 1.5
matplotlib.rcParams['font.size'] = 18
matplotlib.rcParams['xtick.major.size'] = 5
matplotlib.rcParams['xtick.major.width'] = 2
matplotlib.rcParams['ytick.major.size'] = 5
matplotlib.rcParams['ytick.major.width'] = 2
matplotlib.rcParams['figure.figsize'] = (16.0, 8.0)

## Load data

In [ ]:
sr, full_mix = read("../data/full-mix.wav", mmap=False)
sr, only_guitar = read("../data/only-guitar.wav", mmap=False)

full_mix = full_mix.astype(np.float) / 2**15
only_guitar = only_guitar.astype(np.float) / 2**15

## Visualize the full mix

In [ ]:
from scipy.signal import stft
f_vec, t_vec, Zxx = stft(full_mix, sr, nperseg=4096)
plt.pcolormesh(t_vec, f_vec, np.abs(Zxx), vmin=0, vmax=0.5)
# plt.title('STFT Magnitude')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.ylim(0, 3000)
plt.xlim(30, 40);

## Detect where guitar is playing in solo track

In [ ]:
df = pd.DataFrame({'guitar': only_guitar}, index=np.arange(len(only_guitar))/ 44100)
df['full_mix'] = full_mix
df['rolling_mse'] = df['guitar'].rolling(window=2000, center=False).std()
df['is_playing'] = df['rolling_mse'] > 0.1
df['is_playing_float'] = df['is_playing'] * 1.0
df['signal_masked'] = df['is_playing'] * df['guitar']

In [ ]:
t0 = 40
delta = 10
df.loc[t0:t0+delta][['guitar', 'rolling_mse', 'is_playing_float']].plot()
plt.axhline(y=0, color='k')

In [ ]:
play_array(df['signal_masked'].values, range_secs=[20,30])

## Make data set

In [ ]:
def create_random_fragment_indices(n_batches, n_samples, fragment_length):
    i_max = n_samples - fragment_length
    start = np.random.randint(low=0, high=i_max, size=n_batches)
    
    return np.array([np.arange(s, s+fragment_length) for s in start])

In [ ]:
x_train = df['full_mix'].values
y_train = df['is_playing'].values

fragment_length = 4096
n_fragments = 20000

ind = create_random_fragment_indices(n_fragments, len(x_train), fragment_length)

In [ ]:
x_train = x_train[ind]
y_train = y_train[ind]

In [ ]:
x_train = x_train.reshape(-1, fragment_length)
y_train = y_train.reshape(-1, fragment_length)

y_train = np.percentile(y_train, q=80, axis=1).reshape(-1, 1)

# Class balancing
w0 = 0.5*len(x_train) / np.sum(y_train == 0) 
w1 = 0.5*len(x_train) / np.sum(y_train == 1)
sample_weight = np.where(y_train==0, w0, w1).reshape(-1)

## Build model

In [ ]:
import tensorflow as tf

inp = Input(shape=[fragment_length])
out = inp

def apply_fft(tensor):
    tensor = tf.cast(tensor, tf.complex64)
    return tf.abs(tf.spectral.fft(tensor))

out = Lambda(apply_fft, output_shape=[fragment_length])(out)
out = Dense(units=1, activation='sigmoid')(out)

model = Model(inp, out)
model.summary()
model.compile(Adam(lr=1e-3), 'binary_crossentropy', metrics=['accuracy'])

## Fit model

In [ ]:
model.fit(x_train, y_train, epochs=10, validation_split=0.2, sample_weight=sample_weight)

## Make prediction

In [ ]:
df = df.iloc[0:((len(df) // fragment_length) * fragment_length)].copy()
x_pred = df.full_mix.values.reshape(-1, fragment_length)
y_pred = model.predict(x_pred)
df['is_playing_prediction'] = y_pred.repeat(fragment_length)

## Visualize prediction

In [ ]:
import moviepy as mpy
from moviepy.editor import VideoClip, AudioFileClip, clips_array
from moviepy.video.io.bindings import mplfig_to_npimage

In [ ]:
matplotlib.rcParams['figure.figsize'] = (8.0, 4.0)

fig, ax = plt.subplots()
def make_frame1(t):
    ax.clear()
    ix = df.index.searchsorted(t)
    c = '0.5' if df.iloc[ix]['is_playing_prediction'] < 0.5 else 'b'
    ax.plot(df.loc[t-0.2:t]['full_mix'].values, color=c)
    ax.set_ylim(-2, 2)
    return mplfig_to_npimage(fig)

def make_frame2(t):
    ax.clear()
    ix = df.index.searchsorted(t)
    c = '0.5' if df.iloc[ix]['is_playing_prediction'] < 0.5 else 'b'
    ax.plot(df.loc[t-0.2:t]['guitar'].values, color=c)
    ax.set_ylim(-2, 2)
    return mplfig_to_npimage(fig)

def make_frame3(t):
    ax.clear()
    ax.plot(df.loc[t-1:t]['is_playing'].values)
    ax.plot(df.loc[t-1:t]['is_playing_prediction'].values)
    ax.set_ylim(-0.5, 1.5)
    return mplfig_to_npimage(fig)

duration = df.index.max()
# duration = 10

clip1 = VideoClip(make_frame1, duration=duration)
clip2 = VideoClip(make_frame2, duration=duration)
clip3 = VideoClip(make_frame3, duration=duration)

audio = AudioFileClip("../data/full-mix.wav", fps=44100)
clip1.audio = audio

final_clip = clips_array([[clip1, clip2], [clip3, clip3]])
# final_clip.ipython_display(fps=20, autoplay=True, maxduration=300)
final_clip.to_videofile("detection.mp4", fps=24)